<a href="https://colab.research.google.com/github/s528661/nlp100/blob/main/nlp100_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 80. ID番号への変換

問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [1]:
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 3.1 MB/s eta 0:00:00


In [2]:
import re,sys,os

import pickle
import time
import copy

import math
import random
import string

from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import scipy as scp

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn import feature_extraction, preprocessing

from sklearn import svm
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, f1_score
from sklearn.metrics import accuracy_score,  precision_score, recall_score, confusion_matrix, roc_auc_score

# import dask.array as da

import lightgbm as lgb

# Neural Language Processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest

from sklearn.linear_model import RidgeClassifier
import MeCab

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip --no-check-certificate
!unzip ./NewsAggregatorDataset.zip

--2023-12-10 08:52:31--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas     [    <=>             ]  27.87M  35.8MB/s    in 0.8s    

2023-12-10 08:52:33 (35.8 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203]

Archive:  ./NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [4]:
data = pd.read_csv('newsCorpora.csv',delimiter='\t',header=None,
                 names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])

In [5]:
fix_data = data[data['PUBLISHER'].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]

In [6]:
def preprocessing(text):
  text = "".join([i for i in text if i not in string.punctuation])
  text = text.lower()
  text = re.sub("[0-9]+", "", text)
  return text

In [7]:
fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))

<ipython-input-7-0330034268ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))


In [8]:
train_all, test = train_test_split(fix_data, test_size=0.1, shuffle=True)

In [9]:
train, val = train_test_split(train_all, test_size=0.1, shuffle=True)

In [10]:
train.to_csv("train.txt", sep="\t", index=False, header=None)
val.to_csv("valid.txt", sep="\t", index=False, header=None)
test.to_csv("test.txt", sep="\t", index=False, header=None)

In [11]:
def preprocessing(text):
  text = "".join([i for i in text if i not in string.punctuation])
  text = text.lower()
  text = re.sub("[0-9]+", "", text)
  return text

In [12]:
fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))

<ipython-input-12-0330034268ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_data['pTITLE'] = fix_data['TITLE'].apply(lambda x: preprocessing(x))


In [13]:
X = fix_data['pTITLE']

In [14]:
X_train_all, X_test = train_test_split(X, test_size=0.1, shuffle=True)

In [15]:
X_train, X_val = train_test_split(X_train_all, test_size=0.1, shuffle=True)

In [16]:
X_train

207824    china april fiscal revenues up  percent yearon...
268363    update automakers report strong uptick in may ...
85515     disney chief reveals star warsepisode vii has ...
71100     inflation drop pins peripheral bond yields at ...
275668    euro needs to fall to  to satisfy ecb strategists
                                ...                        
2207      orlando gun show canceled after george zimmerm...
297226    global marketsdollar rises on higher us inflat...
107506    diners may have been exposed to hepatitis a at...
373623    all five of garth brooks irish comeback shows ...
287954    siemens said to prepare alstom offer to gain g...
Name: pTITLE, Length: 10805, dtype: object

In [17]:
X_val

98953     woody allen under fire for allwhite cast of br...
83946     global marketsjapan tech shares slip europe se...
391360    update takata says to book  mln in special los...
76218     update duratas antiinfective drug shows effica...
403139    chris pratt  chris pratt embarrassed by weight...
                                ...                        
57169     forexeuro weak near week lows as ecb tries to ...
37842     us stocks drop on slowing factory data as biot...
308393    dem mayors in red states to fight climate chan...
255566    the ftc wants data brokers to share what they ...
43042     home  kanye west  kanye west found the next ri...
Name: pTITLE, Length: 1201, dtype: object

In [18]:
X_test

203304    chinas oil rig move leaves vietnam others look...
362066    could justin bieber be behind selena gomez  de...
100972         you owe the irs dont panic  you have options
232495    why im giving it all to the national september...
33766     anna wintour  anna wintour defends kim kardash...
                                ...                        
290463    correctedalibaba names partnership members in ...
216769    deutsche bank prepares  bln euro capital hike ...
157070      ariel castro victims demand joan rivers apology
344168     adp says companies in us add most workers since 
366806    update air france warns on profit as overcapac...
Name: pTITLE, Length: 1334, dtype: object

In [19]:
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [20]:
words = {}
for title in X_train:
  for word in title.rstrip().split():
    words[word] = 0

In [21]:
for title in X_train:
  for word in title.rstrip().split():
    words[word] += 1

In [22]:
id_words = [(idx, item) for idx, item in enumerate(words.items()) if item[1] >= 2]

In [23]:
id_words = sorted(id_words, key=lambda x: x[1][1], reverse=True)

In [24]:
id_words

[(38, ('to', 2922)),
 (12, ('in', 1916)),
 (84, ('the', 1606)),
 (90, ('of', 1464)),
 (101, ('for', 1348)),
 (72, ('on', 1332)),
 (61, ('as', 1141)),
 (7, ('update', 1036)),
 (14, ('us', 1022)),
 (110, ('and', 989)),
 (88, ('a', 827)),
 (197, ('with', 763)),
 (33, ('at', 664)),
 (144, ('after', 606)),
 (111, ('is', 602)),
 (150, ('new', 551)),
 (129, ('says', 422)),
 (331, ('from', 400)),
 (4, ('up', 376)),
 (388, ('by', 334)),
 (195, ('kardashian', 271)),
 (205, ('over', 255)),
 (287, ('her', 251)),
 (1251, ('kim', 242)),
 (224, ('be', 237)),
 (66, ('stocks', 226)),
 (114, ('more', 220)),
 (0, ('china', 218)),
 (13, ('may', 213)),
 (210, ('first', 212)),
 (632, ('will', 200)),
 (233, ('shares', 199)),
 (686, ('data', 199)),
 (339, ('about', 194)),
 (83, ('are', 192)),
 (36, ('euro', 188)),
 (1253, ('out', 186)),
 (778, ('it', 181)),
 (599, ('not', 180)),
 (41, ('ecb', 177)),
 (15, ('sales', 172)),
 (158, ('that', 171)),
 (1003, ('deal', 167)),
 (1018, ('miley', 166)),
 (1019, ('cyrus'

# 81. RNNによる予測  

In [25]:
dw = 300
dh = 50
class RNN(nn.Module):
  def __init__(self,dw,dh,n_word,n_label):
    super().__init__()
    self.rnn = nn.RNN(dw,dh,nonlinearity="tanh")
    self.emb = nn.Embedding(n_word,dw)
    self.layer = nn.Linear(dh,n_label,bias=True)
    self.softmax = nn.Softmax()

  def forward(self,x,h):
    x = self.emb(x)
    x, h = self.rnn(x,h=None)
    x = x[:,-1,:]
    x = self.softmax(x)
    return x

# 82. 確率的勾配降下法による学習  
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題81で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

In [31]:
def word2vec(data):
  vectorizer = TfidfVectorizer() # 1-gram, 2-gramでTfidfを計算
  vec = vectorizer.fit_transform(data).toarray()
  vecs = pd.DataFrame(vec, columns=vectorizer.get_feature_names_out())
  return vecs

In [27]:
def word2id(data):
  words = {}
  for title in data:
    for word in title.rstrip().split():
      words[word] = 0

  for title in data:
    for word in title.rstrip().split():
      words[word] += 1

  id_words = [(idx, item) for idx, item in enumerate(words.items()) if item[1] >= 2]
  id_words = sorted(id_words, key=lambda x: x[1][1], reverse=True)

  return id_words

In [28]:
import itertools

In [33]:
datasets = []
for data in [X_train,X_val,X_test]:
  id_data = word2id(list(itertools.chain.from_iterable([item.split() for item in data])))
  #print(id_data)
  id_data = np.array([item[0] for idx, item in id_data])
  print(id_data)
  vec_data = word2vec(id_data)
  datasets.append(vec_data)

['to' 'in' 'the' ... 'wyman' 'marking' 'rubin']
['to' 'in' 'for' ... 'tina' 'fey' 'hewitt']
['to' 'in' 'the' ... 'sarandon' 'fossil' 'friends']
